In [47]:
import os
import cv2
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [87]:
ANNOTATIONS_PATH = 'annotations/annotations.xml'
IMAGE_FOLDER_PATH = "Fetus/"
GT_FOLDER_PATH = "Fetus_GT_Russel/"

try:
    os.makedirs(GT_FOLDER_PATH)
except:
    pass

In [88]:
tree = ET.parse(ANNOTATIONS_PATH)
root = tree.getroot()

In [89]:
root.tag

'annotations'

In [90]:
root.attrib

{}

In [91]:
for child in root:
    if len(child.attrib) > 0:
        img_name = child.attrib['name']
#         print(child.attrib)
        img_path = IMAGE_FOLDER_PATH+img_name
        img = cv2.imread(img_path , cv2.IMREAD_GRAYSCALE)
        mask = np.zeros(img.shape , dtype = np.uint8)
    
        for childs_child in child: 
            if len(childs_child.attrib) > 0:
    #             print(childs_child.attrib)
                points = childs_child.attrib['points']
                points = points.split(';')
                points = [point.split(',') for point in points] 
                points = [[int(float(point[0])), int(float(point[1]))] for point in points]
                points = np.array(points, dtype=np.int32)
                cv2.fillPoly(mask, [points], 255)
                gt_mask = mask.copy()
                gt_mask = np.array(gt_mask, np.uint8)
                contours, _ = cv2.findContours(gt_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                convex_hulls = []
                for contour in contours:
                    convex_hull = cv2.convexHull(contour)
                    convex_hulls.append(convex_hull)
                cv2.drawContours(gt_mask, convex_hulls, -1, 1, 1)
#                 plt.imshow(img)
#                 plt.show()
                mask_name = GT_FOLDER_PATH+str(img_name.split('.')[0])+".png"
                cv2.imwrite(mask_name, gt_mask)
                break
        break